In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

# CHAIN with Parallel Chains

In [2]:
# task - 1 [prompt]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a TV Series summarizer."),
    ("human", "Please summarize the following TV series in brief : {input}")
])

In [3]:
# task - 2 [LLM]

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

In [4]:
# task - 3 [Str parser]

parser = StrOutputParser()

In [6]:
# task - 4 [Custom Runnable]

from langchain_core.runnables import RunnableLambda

def dictionary_maker(text:str)-> dict:
    
    return {"text": text}

dictionary_maker_runnable = RunnableLambda(dictionary_maker)

### Parallel Chain 1

In [10]:
# task - 1 [prompt]

linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator."),
    ("human", "Create a post for the following text for LinkedIn : {text}")
])

# task - 2 [LLM]

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

# task - 3 [Str parser]

parser = StrOutputParser()


chain_linkedin = linkedin_prompt | llm | parser

### Parallel Chain 2

In [8]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

In [7]:
def insta_chain(text:dict):

    text = text["text"]

    # task - 1 [prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a Instagram post generator."),
        ("human", "Create a post for the following text for Instagram : {text}")
        ])

    # task - 2 [LLM]
    llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

    # task - 3 [Str parser]
    parser = StrOutputParser()
    
    chain_insta = insta_prompt | llm | parser

    result = chain_insta.invoke(text)

    return result

insta_chain_runnable = RunnableLambda(insta_chain)

## Final Orchestration

In [11]:
final_chain = (
    prompt_template | 
    llm | 
    parser | 
    dictionary_maker_runnable | 
    RunnableParallel(branches = {"linkedin": chain_linkedin, "instagram": insta_chain_runnable})
)

In [12]:
final_chain.invoke("Better Call Saul")

{'branches': {'linkedin': 'Here\'s a LinkedIn-style post based on the provided text:\n\n**The Transformation of a Leader: Lessons from "Better Call Saul"**\n\nAs professionals, we\'ve all been faced with tough decisions that challenge our values and morals. The hit series "Better Call Saul" offers a fascinating case study in the transformation of a small-time lawyer, Jimmy McGill, into the morally ambiguous lawyer Saul Goodman.\n\nThrough Jimmy\'s journey, we see the complexities of human nature and the gray areas that exist in our personal and professional lives. His struggles with his brother Chuck, a successful lawyer with electromagnetic hypersensitivity, and his friend Kim Wexler, a fellow lawyer, serve as a reminder that our relationships and environments can greatly impact our decisions and actions.\n\nAs Jimmy navigates the legal world and becomes embroiled in organized crime, we\'re forced to confront the consequences of our choices and the blurred lines between right and wron

# Chain as a Runnable

In [15]:
# task - 1 [Beautify Function]

def beautify(final_response:dict)-> dict:

    linkedin_response = final_response["branches"]["linkedin"]
    instagram_response = final_response["branches"]["instagram"]

    return {"linkedin": linkedin_response, "instagram": instagram_response}

beautify_runnable = RunnableLambda(beautify)

# task - 2 [Final Chain]

beautified_chain = final_chain | beautify_runnable

beautified_chain.invoke("Rings of the power")

{'linkedin': 'Here\'s a LinkedIn-style post based on the provided text:\n\n**Immersing Ourselves in the World of Fantasy: Lessons from "Rings of Power"**\n\nAs professionals, we\'re often drawn to stories that inspire and captivate us. The new fantasy drama TV series, "Rings of Power", is a great example of this. Based on J.R.R. Tolkien\'s iconic novels, "The Lord of the Rings" and "The Hobbit", this show transports us to the Second Age of Middle-earth, a world of wonder and magic.\n\nWhat can we learn from this epic tale? Here are a few takeaways:\n\n **The Power of Alliances**: The series highlights the importance of forming strong alliances and collaborations to overcome adversity. In our own careers, we can apply this lesson by building diverse networks and partnerships to drive success.\n\n **Innovation and Creativity**: The show\'s intricate world-building and magical landscapes demonstrate the value of imagination and creativity in shaping our work and industries.\n\n **Resilien